In [51]:
import pandas as pd
import os
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image
from torch import nn

In [80]:
class LegoDataset(Dataset):
    """Custom dataset class for Lego Minifigure dataset."""
    
    def __init__(self, img_dir, test=False, transform=None, target_transform=None):
        """init method.
        
        Keyword arguments:
        img_dir -- the path to the root image directory of test and train data
        test -- True to load test data (default: False)
        transform -- transform to apply to X
        target_transform -- transform to apply to y"""
        
        self.transform = transform
        self.target_transform = target_transform
        self.test = test
        self.img_dir = os.path.join(img_dir, "test/" if test else "train/")
        self.full_df = None
        
        # read path names and class names from csv files
        meta_df = pd.read_csv(os.path.join(self.img_dir, "metadata.csv"))
        if self.test:
            test_df = pd.read_csv(os.path.join(self.img_dir, "test.csv"))
            self.full_df = test_df.merge(meta_df, on="class_id")
        else:
            train_df = pd.read_csv(os.path.join(self.img_dir, "index.csv"))
            self.full_df = train_df.merge(meta_df, on="class_id")
    
    def __len__(self):
        return len(self.full_df)
    
    def __getitem__(self, idx):
        row = self.full_df.iloc[idx]
        image = read_image(os.path.join(self.img_dir, row["path"])).float()
        label = row["class_id"] - 1
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [81]:
# load datasets for getting mean and std for standardization
def get_mean_std(size=[224, 224]):
    transform = torchvision.transforms.Compose([
        torchvision.transforms.Resize(size)
    ])
    train_dataset = LegoDataset("data/", test=False, transform=transform)

    # get mean and std of dataset
    loader = DataLoader(train_dataset, batch_size=len(train_dataset))
    data = next(iter(loader))
    return data[0].mean(), data[0].std()

In [82]:
# prep data for real
input_size = [64, 64]
mean, std = get_mean_std(input_size)
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize(input_size),
    torchvision.transforms.Normalize(mean, std)
])
train_dataset = LegoDataset("data/", test=False, transform=transform)
test_dataset = LegoDataset("data/", test=True, transform=transform)


In [90]:
# simple conv net
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        
        # conv_stack
        self.conv_stack = nn.Sequential(
            nn.Conv2d(3, 16, 5),
            nn.ReLU(),
            nn.MaxPool2d((2, 2)),
            nn.Conv2d(16, 32, 3),
            nn.ReLU(),
            nn.MaxPool2d((2, 2)),
            nn.Conv2d(32, 32, 3),
            nn.ReLU(),
            nn.MaxPool2d((2, 2))
        )
        
        self.flatten = nn.Flatten()
        
        # 2 dense layers
        self.dense_stack = nn.Sequential(
            nn.Linear(1152, 100),
            nn.ReLU(),
            nn.Linear(100, 36)
        )
        
    
    def forward(self, x):
        x = self.conv_stack(x)
        x = self.flatten(x)
        x = self.dense_stack(x)
        return x
    
    def train(self, optimizer, loss_fn, train_loader, test_loader, n_epochs):
        for e in range(n_epochs):
            total = 0
            correct = 0
            for X, y in train_loader:
                # get preds
                preds = self.forward(X)
                loss = loss_fn(preds, y)
                
                # backprop
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
                # collect stats
                max_preds = preds.argmax(1)
                total += y.size()[0]
                correct += sum(max_preds == y)
            test_total = 0
            test_correct = 0
            
            # get test acc
            with torch.no_grad():
                for X, y in test_loader:
                    preds = model(X)
                    max_preds = preds.argmax(1)
                    test_total += y.size()[0]
                    test_correct += sum(max_preds == y)
            
            train_acc = correct / total
            test_acc = test_correct / test_total
            # print out stats
            print(f"epoch: {e}\t train_acc: {train_acc:.3f}\t test_acc: {test_acc:.3f}")
                
                

In [91]:
# run everything!
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=16)
test_loader = DataLoader(test_dataset, shuffle=True, batch_size=16)
model = ConvNet()

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [92]:
model.train(optimizer, loss_fn, train_loader, test_loader, 50)

epoch: 0	 train_acc: 0.040	 test_acc: 0.068
epoch: 1	 train_acc: 0.065	 test_acc: 0.041
epoch: 2	 train_acc: 0.128	 test_acc: 0.096
epoch: 3	 train_acc: 0.222	 test_acc: 0.123
epoch: 4	 train_acc: 0.372	 test_acc: 0.205
epoch: 5	 train_acc: 0.537	 test_acc: 0.178
epoch: 6	 train_acc: 0.636	 test_acc: 0.178
epoch: 7	 train_acc: 0.770	 test_acc: 0.247
epoch: 8	 train_acc: 0.852	 test_acc: 0.205
epoch: 9	 train_acc: 0.881	 test_acc: 0.233
epoch: 10	 train_acc: 0.938	 test_acc: 0.205
epoch: 11	 train_acc: 0.960	 test_acc: 0.247
epoch: 12	 train_acc: 0.960	 test_acc: 0.219
epoch: 13	 train_acc: 0.983	 test_acc: 0.301
epoch: 14	 train_acc: 0.994	 test_acc: 0.260
epoch: 15	 train_acc: 0.997	 test_acc: 0.247
epoch: 16	 train_acc: 0.997	 test_acc: 0.288
epoch: 17	 train_acc: 1.000	 test_acc: 0.274
epoch: 18	 train_acc: 0.997	 test_acc: 0.274
epoch: 19	 train_acc: 0.997	 test_acc: 0.274
epoch: 20	 train_acc: 1.000	 test_acc: 0.315
epoch: 21	 train_acc: 1.000	 test_acc: 0.274
epoch: 22	 train_acc